# Wordle

In [ ]:
import collections
import copy
import pathlib
import random
import string
from typing import Optional, TypedDict

import jupyter_black
import pandas as pd

jupyter_black.load(lab=False, line_length=79)

## Build the lookup tables

Read possible guesses (from https://gist.github.com/cfreshman).

In [ ]:
words = pathlib.Path("data/wordle-nyt-words-14855.txt").read_text().split()
assert all(len(word) == 5 for word in words)
assert len(set(words)) == len(words)
assert all(word.islower() for word in words)

Iterate over words and populate lookups

In [ ]:
contains: dict[str, set[str]] = collections.defaultdict(lambda: set())
does_not_contain: dict[str, set[str]] = collections.defaultdict(lambda: set())
contains_at: dict[tuple[str, int], set[str]] = collections.defaultdict(
    lambda: set()
)

alphabet = set(string.ascii_lowercase)

for word in words:
    for position, letter in enumerate(word):
        contains[letter].add(word)
        contains_at[(letter, position)].add(word)
    for letter in alphabet - set(word):
        does_not_contain[letter].add(word)

contains_not_at: dict[tuple[str, int], set[str]] = {}

for letter in string.ascii_lowercase:
    for position in range(5):
        contains_not_at[(letter, position)] = (
            contains[letter] - contains_at[(letter, position)]
        )

for lookup in [contains, does_not_contain, contains_at, contains_not_at]:
    assert set.union(*(lookup.values())) == set(words)

In [ ]:
class Lookup(TypedDict):
    contains_at: dict[tuple[str, int], set[str]]
    does_not_contain: dict[str, set[str]]
    contains_not_at: dict[tuple[str, int], set[str]]


lookup: Lookup = {
    "contains_at": contains_at,
    "does_not_contain": does_not_contain,
    "contains_not_at": contains_not_at,
}

## Play

Function to return candidate solutions given a hint

In [ ]:
def find_candidates(
    hint: str,
    incorrect_positions: Optional[set[int]] = None,
    lookup: Lookup = lookup,
) -> set[str]:
    """Return possible Wordle solutions given a hint.

    hint is a 5-letter word specification. Use uppercase letters
    to indicate letters in the correct position (green) and
    lowercase letters to indicate letters that are either in the
    incorrect position (orange) or do not exist in the word
    (dark grey). Use incorrect_positions to indicate the indices
    in the hint for the letters that exist in the word but are
    incorrectly positioned (i.e., color letters orange using
    these indices).

    For example, hint = 'Tseta' and incorrect_positions = set([1, 3])
    specify that the word starts with a T, contains s and another
    t but in incorrect positions and does not contain an a.
    """

    if incorrect_positions is None:
        incorrect_positions = set()

    if len(hint) != 5:
        raise ValueError(f"hint: {hint} must be of length 5")
    if len(incorrect_positions) > 5:
        raise ValueError("length of incorrect_positions must be at most 5")
    for incorrect_position in incorrect_positions:
        if not (0 <= incorrect_position < 5):
            raise ValueError(
                f"incorrect position {incorrect_position} out of bounds"
            )

    word_sets = []
    letters_in_incorrect_positions = set(
        [hint[position] for position in incorrect_positions]
    )
    letters_in_correct_positions = set(
        [
            hint[position].lower()
            for position, letter in enumerate(hint)
            if letter.isupper()
        ]
    )
    for position, letter in enumerate(hint):
        upper_letter = letter.isupper()
        position_in_incorrect_positions = position in incorrect_positions
        if upper_letter and position_in_incorrect_positions:
            raise ValueError(
                "correct letter (upper) position cannot be in incorrect_positions"
            )
        elif upper_letter:
            word_sets.append(lookup["contains_at"][(letter.lower(), position)])
        elif position_in_incorrect_positions:
            word_sets.append(lookup["contains_not_at"][(letter, position)])
        elif letter not in letters_in_incorrect_positions.union(
            letters_in_correct_positions
        ):
            word_sets.append(lookup["does_not_contain"][letter])

    if len(incorrect_positions) == 5:
        for letter in alphabet - set(hint.lower()):
            word_sets.append(lookup["does_not_contain"][letter])
    return set.intersection(*word_sets)


assert find_candidates("HELPS") == set(["helps"])
assert find_candidates("AWAek", incorrect_positions=[3, 4]) == set(["awake"])
assert find_candidates("HEplS", incorrect_positions=set([2, 3])) == set(
    ["helps"]
)
assert find_candidates("Gohts", incorrect_positions=set([1, 2, 3, 4])) == set(
    ["ghost"]
)
assert find_candidates("pPael", incorrect_positions=set([0, 2, 3, 4])) == set(
    ["apple"]
)
assert find_candidates(
    "elapp", incorrect_positions=set([0, 1, 2, 3, 4])
) == set(["appel", "apple", "lapel", "palea", "pepla"])
assert find_candidates("praps", incorrect_positions=[0, 1, 2, 3, 4]) == set([])
assert find_candidates("hsARE", incorrect_positions=[0, 1]) == set(["share"])
assert find_candidates("APPLc") == set(["apple", "apply"])
assert find_candidates("AddLE") == set(
    [
        "abele",
        "agile",
        "ahole",
        "aisle",
        "aizle",
        "amble",
        "amole",
        "ample",
        "ancle",
        "anele",
        "angle",
        "anile",
        "ankle",
        "anole",
        "apple",
        "argle",
        "avale",
        "axile",
        "azole",
    ]
)
assert find_candidates("ApsiS", set([1])) == set(
    ["aapas", "alaps", "arpas", "ataps"]
)

Function to return a hint given an answer and a guess

In [ ]:
def give_hint(guess: str, answer: str) -> tuple[str, set[int]]:
    """Get a hint and incorrect letter positions for a guess and answer."""

    guess = guess.lower()
    answer = answer.lower()

    if len(guess) != 5:
        raise ValueError(f"guess must be of length 5")
    if len(answer) != 5:
        raise ValueError(f"answer must be of length 5")
    if not guess.isalpha():
        raise ValueError("guess must be alphabetic")
    if not answer.isalpha():
        raise ValueError("answer must be alphabetic")

    answer_positions = list(range(5))
    correct_positions = set([])
    incorrect_positions = set([])

    for guess_position, guess_letter in enumerate(guess):
        if answer[guess_position] == guess_letter:
            answer_positions.remove(guess_position)
            correct_positions.add(guess_position)

    for guess_position, guess_letter in enumerate(guess):
        if guess_position in correct_positions:
            continue
        found_orange = False
        for answer_position in answer_positions:
            if answer[answer_position] == guess_letter:
                found_orange = True
                break
        if found_orange:
            incorrect_positions.add(guess_position)
            answer_positions.remove(answer_position)

    hint = ""
    for i in range(5):
        if i in correct_positions:
            hint += guess[i].upper()
        else:
            hint += guess[i]
    return hint, incorrect_positions


assert give_hint("speak", "stays") == ("Speak", set([3]))
assert give_hint("speak", "speak") == ("SPEAK", set([]))
assert give_hint("leapp", "apple") == ("leapp", set([0, 1, 2, 3, 4]))
assert give_hint("eplap", "apple") == ("ePlap", set([0, 2, 3, 4]))

Function to play a game and return the number of guesses to a solution

In [ ]:
def game(
    answer: str,
    words: set[str] = set(words),
    lookup: Lookup = lookup,
    seed: Optional[int] = 42,
) -> int:
    random.seed(seed)
    possible_answers = set([])
    num_guesses = 0
    candidates = copy.copy(words)
    hint = ""
    while hint.lower() != answer:
        guess = random.choice(list(candidates))
        num_guesses += 1
        hint, incorrect_positions = give_hint(guess, answer)
        candidates.remove(guess)
        candidates.intersection_update(
            find_candidates(hint, incorrect_positions, lookup=lookup)
        )
    return num_guesses

Play all Wordles

In [ ]:
%%time
# takes about 30 seconds
guesses = []
for word in words:
    guesses.append(game(word))
# not reproducible even with seed, possibly because
# of the ordering in the sets changing between runs

## Analyse

Average guesses to win

In [ ]:
s = pd.Series(guesses)
s.mean()

Percentage of games won ($\leq$ 5 guesses)

In [ ]:
sum(s < 6) / len(s) * 100

In [ ]:
s.hist();